In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-geometric
!pip install -q git+https://github.com/snap-stanford/deepsnap.git

!pip install ogb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import pandas as pd
import shutil, os
import os.path as osp
import numpy as np

import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU

from torch_geometric.utils import negative_sampling, to_dense_adj
from torch_geometric.data import Data#, DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GINConv, GATConv
from torch_sparse import SparseTensor
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

#from logger import Logger
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

Device: cuda


In [53]:
def get_data(path, picky=False):
            mode='TEST'
            df=pd.read_csv(path+mode+'annotated_links.csv', names=['source', 'target', 'link'])#.drop('link', axis=1)
            if picky==True:
              df=df[df['link']==1].drop('link', axis=1)
            else:
               df=df[df['link']!=0].drop('link', axis=1)
            #df.index+=1
            #node_ids = df.index
            #link = pd.read_csv(path+mode+'annotated_links.csv', names=['source', 'target', 'link'])#['link']
            #link.index+=1

            cl=pd.read_csv(path+mode+'colours.csv')
            #cl.index+=1
            di=cl['ImgName'].to_dict()
            node2index = {v: k for k, v in di.items()}
            df=df.replace({"source": node2index}).replace({"target": node2index})
            cl=cl.drop('ImgName', axis=1)
            
            node_feats= torch.tensor(cl.to_numpy(), dtype=torch.float)
            num_nodes=len(node_feats)
            
            #edge_feats = torch.tensor(link['link'].to_numpy(), dtype=torch.float)
            
            #edges = link.loc[link['source'].isin(node_ids)]
            edge_index = torch.tensor(df.to_numpy(), dtype=torch.long).T
            
            data = Data(x=node_feats, edge_index=edge_index,  node2index=node2index, num_nodes=num_nodes)
            return data

def get_data_split(path, picky=False):
    data = get_data(path, picky)
    transform = RandomLinkSplit(is_undirected=True)
    train_data, val_data, test_data = transform(data)
    train_data.edge_index=train_data.edge_index.long()
    val_data.edge_index=val_data.edge_index.long()
    test_data.edge_index=test_data.edge_index.long()
    return train_data, val_data, test_data

Anzahl der edges ungerade -> nur je ein Paar für undirected edges

In [54]:
path='/content/drive/MyDrive/data/GNNrawData/'
mode="TEST"
PKY=True

In [55]:
dota = get_data(path, picky=PKY)
dota.edge_index=dota.edge_index.long()

In [56]:
adj_t = SparseTensor(row=dota.edge_index[0], col=dota.edge_index[1],
                   sparse_sizes=(dota.num_nodes, dota.num_nodes))
full_adj_t = to_dense_adj(dota.edge_index)

In [57]:
from torch_geometric.utils import train_test_split_edges

path = '/content/drive/MyDrive/data/GNNrawData/'
dataT = get_data(path, picky=PKY)
dataT.edge_index=dataT.edge_index.long()
# train_data, val_data, test_data = get_data_split(path, picky=PKY)
dataT = train_test_split_edges(dataT)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [58]:
dota.adj_t=adj_t
dota.full_adj_t = to_dense_adj(dota.edge_index)

## RandomLinkSpit

In [59]:
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(is_undirected=True, split_labels=True, add_negative_train_samples =True)
train_data, val_data, test_data = transform(dota)

In [60]:
train_data.edge_index

tensor([[ 6,  0,  4, 18,  7,  1,  7,  3,  9,  0,  7, 13,  8, 15,  1,  4, 10,  1,
          0, 10,  8, 15,  9,  7, 14,  8,  5, 17, 13,  0,  3, 19,  5,  0,  4,  4,
         10, 14, 17, 12,  7,  4, 18, 20, 10, 20, 17, 18, 11, 11, 18,  1, 18, 14,
         17, 17,  7, 18, 20, 19, 19, 11, 14, 19, 10, 19, 17, 15, 15, 19, 16, 18,
          4, 20, 17,  3,  9,  8, 12, 15, 18, 19, 10,  6],
        [18, 20, 10, 20, 17, 18, 11, 11, 18,  1, 18, 14, 17, 17,  7, 18, 20, 19,
         19, 11, 14, 19, 10, 19, 17, 15, 15, 19, 16, 18,  4, 20, 17,  3,  9,  8,
         12, 15, 18, 19, 10,  6,  6,  0,  4, 18,  7,  1,  7,  3,  9,  0,  7, 13,
          8, 15,  1,  4, 10,  1,  0, 10,  8, 15,  9,  7, 14,  8,  5, 17, 13,  0,
          3, 19,  5,  0,  4,  4, 10, 14, 17, 12,  7,  4]])

In [61]:
train_data.adj_t =  SparseTensor(row=train_data.edge_index[0], col=train_data.edge_index[1],
                   sparse_sizes=(train_data.num_nodes, train_data.num_nodes))
train_data.full_adj_t = to_dense_adj(train_data.edge_index)

test_data.adj_t =  SparseTensor(row=test_data.edge_index[0], col=test_data.edge_index[1],
                   sparse_sizes=(test_data.num_nodes, test_data.num_nodes))
test_data.full_adj_t = to_dense_adj(test_data.edge_index)

val_data.adj_t =  SparseTensor(row=val_data.edge_index[0], col=val_data.edge_index[1],
                   sparse_sizes=(val_data.num_nodes, val_data.num_nodes))
val_data.full_adj_t = to_dense_adj(val_data.edge_index)

In [62]:
train_data=train_data.to(device) 
test_data=test_data.to(device) 
val_data=val_data.to(device) 

# Define models

In [63]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

class GIN(torch.nn.Module):
    def __init__(self, in_channels, dim, out_channels, num_layers,
                 dropout):
        super().__init__()
        
        self.conv1 = GINConv(
            Sequential(Linear(in_channels, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU())) # GINConv takes a neural network as input

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))
        
        self.dropout = dropout
  
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()


    def forward(self, x, adj_t):
        x = self.conv1(x, adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, adj_t)
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

# Training functions

In [64]:
def train(model, predictor, optimizer, batch_size):
    model.train()
    predictor.train()
    pos_train_edge = train_data.pos_edge_label_index.T.to(train_data.x.device)   # HIER ----------------------------------------- #
    neg_train_edge = train_data.neg_edge_label_index.T.to(train_data.x.device) 

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(train_data.x, train_data.adj_t)

        edge = pos_train_edge[perm].t()       # HIER ----------------------------------------- #

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        # Instead of negative train edges one could do negative sampling
        edge = neg_train_edge[perm].t()
        
        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, batch_size):
    model.eval()
    predictor.eval()

    h = model(data.x, data.adj_t)
    pos_train_edge = train_data.pos_edge_label_index.T.to(train_data.x.device)# HIER ----------------------------------------- #
    neg_train_edge = train_data.neg_edge_label_index.T.to(train_data.x.device)
    pos_valid_edge = val_data.pos_edge_label_index.T.to(val_data.x.device)# HIER ----------------------------------------- #
    neg_valid_edge = val_data.neg_edge_label_index.T.to(val_data.x.device)# HIER ----------------------------------------- #
    pos_test_edge = test_data.pos_edge_label_index.T.to(test_data.x.device)# HIER ----------------------------------------- #
    neg_test_edge = test_data.neg_edge_label_index.T.to(test_data.x.device)# HIER ----------------------------------------- #

    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    neg_train_preds = []
    for perm in DataLoader(range(neg_train_edge.size(0)), batch_size):
        edge = neg_train_edge[perm].t()
        neg_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_train_pred = torch.cat(neg_train_preds, dim=0)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)


    #print(pos_train_pred)

    # ROC AUC
    results=[]  

    # TRAIN 
    pos_train_labels = pos_train_edge.size(0)
    neg_train_labels = neg_train_edge.size(0)
    Etr = pos_train_labels + neg_train_labels

    link_labels = torch.zeros(Etr, dtype=torch.float, device=device)
    link_labels[:pos_train_labels] = 1

    pos_train_preds=torch.cat(pos_train_preds, dim=0)
    neg_train_preds=torch.cat(neg_train_preds, dim=0)
    link_probs =torch.cat((pos_train_preds,neg_train_preds), dim=0) 
  
    rocauc_train = roc_auc_score(link_labels.cpu(), link_probs.cpu())


    # TEST  
    pos_test_labels = pos_test_edge.size(0)
    neg_test_labels = neg_test_edge.size(0)
    Et = pos_test_labels + neg_test_labels

    link_labels = torch.zeros(Et, dtype=torch.float, device=device)
    link_labels[:pos_test_labels] = 1

    pos_test_preds=torch.cat(pos_test_preds, dim=0)
    neg_test_preds=torch.cat(neg_test_preds, dim=0)
    link_probs =torch.cat((pos_test_preds,neg_test_preds), dim=0) 
  
    rocauc_test = roc_auc_score(link_labels.cpu(), link_probs.cpu())

    # VAL
    pos_val_labels = pos_valid_edge.size(0)
    neg_val_labels = neg_valid_edge.size(0)
    Ev = pos_val_labels + neg_val_labels

    link_labels = torch.zeros(Ev, dtype=torch.float, device=device)
    link_labels[:pos_val_labels] = 1

    pos_val_preds=torch.cat(pos_valid_preds, dim=0)
    neg_val_preds=torch.cat(neg_valid_preds, dim=0)
    link_probs =torch.cat((pos_val_preds,neg_val_preds), dim=0) 
 
    rocauc_val = roc_auc_score(link_labels.cpu(), link_probs.cpu())

    results.append(('train', rocauc_train,'| test ', rocauc_test, '| val ', rocauc_val))

    return results

In [65]:
def train_model(model, gnn_args, predictor, model_name): 

        
  for run in range(2):
        model.reset_parameters()
        predictor.reset_parameters()
        optimizer = torch.optim.Adam(
            list(model.parameters()) + list(predictor.parameters()),
            lr=gnn_args['lr'])


        for epoch in range(1, 1 + gnn_args['epochs']):
            loss = train(model, predictor, optimizer, gnn_args['batch_size'])

            if epoch % gnn_args['eval_steps'] == 0:
                results = test(model, predictor, gnn_args['batch_size'])


                if epoch % gnn_args['log_steps'] == 0:
                    for result in results:
                        
                        #print(result)
                        print(f'Run: {run + 1:02d}, '
                              f'Epoch: {epoch:02d}, '
                              f'Loss: {loss:.4f}, '
                              f'Test: {result}')
                    print('---')

# Train

In [72]:
gnn_args = { 
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'hidden_size': 10,
    'dropout': 0.5,
    'epochs': 10,
    'weight_decay': 1e-5,
    'lr': 0.005,
    'attn_size': 32,
    'num_layers':2,
    'log_steps':1,
    'eval_steps':1,
    'runs':10,
    'batch_size': 6
}

In [67]:
data = dota.to(device)

In [73]:
model_name = 'sage'
input_size = dota.x.size(1)
sage_model = SAGE(input_size, gnn_args['hidden_size'], gnn_args['hidden_size'], 
              gnn_args['num_layers'], gnn_args['dropout']).to(device)  
predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
train_model(sage_model, gnn_args, predictor, 'sage')

Run: 01, Epoch: 01, Loss: 1.4431, Test: ('train', 0.6757369614512472, '| test ', 0.4861111111111111, '| val ', 0.8055555555555556)
---
Run: 01, Epoch: 02, Loss: 1.3994, Test: ('train', 0.6882086167800454, '| test ', 0.47916666666666663, '| val ', 0.7777777777777779)
---
Run: 01, Epoch: 03, Loss: 1.3826, Test: ('train', 0.7046485260770976, '| test ', 0.5069444444444444, '| val ', 0.8333333333333334)
---
Run: 01, Epoch: 04, Loss: 1.3972, Test: ('train', 0.7040816326530612, '| test ', 0.4930555555555556, '| val ', 0.8333333333333334)
---
Run: 01, Epoch: 05, Loss: 1.3491, Test: ('train', 0.7046485260770975, '| test ', 0.5, '| val ', 0.7777777777777779)
---
Run: 01, Epoch: 06, Loss: 1.3383, Test: ('train', 0.7057823129251701, '| test ', 0.5277777777777777, '| val ', 0.8611111111111112)
---
Run: 01, Epoch: 07, Loss: 1.4151, Test: ('train', 0.7029478458049887, '| test ', 0.5, '| val ', 0.888888888888889)
---
Run: 01, Epoch: 08, Loss: 1.3533, Test: ('train', 0.7148526077097506, '| test ', 0.52

In [70]:
model_name = 'gcn'
input_size = dataT.x.size(1)
gcn_model = GCN(input_size, gnn_args['hidden_size'], gnn_args['hidden_size'], 
              gnn_args['num_layers'], gnn_args['dropout']).to(device)  
predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
train_model(gcn_model, gnn_args, predictor, 'gcn')

Run: 01, Epoch: 02, Loss: 1.3968, Test: ('train', 0.41043083900226757, '| test ', 0.3402777777777778, '| val ', 0.7222222222222223)
---
Run: 01, Epoch: 04, Loss: 1.3873, Test: ('train', 0.6740362811791383, '| test ', 0.4930555555555556, '| val ', 0.888888888888889)
---
Run: 01, Epoch: 06, Loss: 1.3838, Test: ('train', 0.778344671201814, '| test ', 0.5555555555555556, '| val ', 0.9166666666666667)
---
Run: 01, Epoch: 08, Loss: 1.3860, Test: ('train', 0.7936507936507937, '| test ', 0.5625, '| val ', 0.9166666666666667)
---
Run: 01, Epoch: 10, Loss: 1.3802, Test: ('train', 0.8027210884353742, '| test ', 0.5625, '| val ', 0.9166666666666667)
---
Run: 02, Epoch: 02, Loss: 1.3799, Test: ('train', 0.49829931972789115, '| test ', 0.5208333333333334, '| val ', 0.5833333333333334)
---
Run: 02, Epoch: 04, Loss: 1.3799, Test: ('train', 0.5736961451247166, '| test ', 0.5277777777777778, '| val ', 0.6944444444444444)
---
Run: 02, Epoch: 06, Loss: 1.3844, Test: ('train', 0.6950113378684807, '| test '

In [71]:
model_name = 'gin'
input_size = dataT.x.size(1)
gin_model = GIN(input_size, gnn_args['hidden_size'], gnn_args['hidden_size'], 
              gnn_args['num_layers'], gnn_args['dropout']).to(device)  
predictor = LinkPredictor(gnn_args['hidden_size'], gnn_args['hidden_size'], 1,
                          gnn_args['num_layers'], gnn_args['dropout']).to(device)
train_model(gin_model, gnn_args, predictor, 'gin')

Run: 01, Epoch: 02, Loss: 1.3901, Test: ('train', 0.7930839002267575, '| test ', 0.7013888888888888, '| val ', 1.0)
---
Run: 01, Epoch: 04, Loss: 1.3252, Test: ('train', 0.782312925170068, '| test ', 0.6805555555555556, '| val ', 1.0)
---
Run: 01, Epoch: 06, Loss: 1.2081, Test: ('train', 0.774092970521542, '| test ', 0.6736111111111112, '| val ', 1.0)
---
Run: 01, Epoch: 08, Loss: 1.1500, Test: ('train', 0.7570861678004536, '| test ', 0.6180555555555556, '| val ', 1.0)
---
Run: 01, Epoch: 10, Loss: 1.1373, Test: ('train', 0.7534013605442177, '| test ', 0.6284722222222222, '| val ', 1.0)
---
Run: 02, Epoch: 02, Loss: 1.4019, Test: ('train', 0.780045351473923, '| test ', 0.625, '| val ', 1.0)
---
Run: 02, Epoch: 04, Loss: 1.3807, Test: ('train', 0.7658730158730159, '| test ', 0.6249999999999999, '| val ', 1.0)
---
Run: 02, Epoch: 06, Loss: 1.3739, Test: ('train', 0.7573696145124719, '| test ', 0.6666666666666666, '| val ', 1.0)
---
Run: 02, Epoch: 08, Loss: 1.3123, Test: ('train', 0.7318